In [43]:
from urllib.request import *
from urllib import request
import urllib.parse
from bs4 import BeautifulSoup
import bs4
import re
import numpy as np
import requests
from tika import parser as tkparser
import lxml
from selenium import webdriver
import time
import pandas as pd
import Exception

ModuleNotFoundError: No module named 'Exception'

In [95]:
class crawler():
    def __init__(self):
        self.papers = pd.DataFrame(columns=['title', 'keywords_lang', 'keywords', 'abstract_lang','abstract'])
        self.df_idx = 0 
        self.max_page_view = 20
        self.basic_url = 'http://kmbase.medric.or.kr'
        self.browser = webdriver.Chrome('C:/Users/NB-0242/jupyter/PHR/chromedriver.exe')
        self.browser.get('http://kmbase.medric.or.kr/Main.aspx?menu=01&d=SEARCH&s=NORMAL&kw=%ub300%uc0ac%uc99d%ud6c4%uad70')
        
#         search_html = self.browser.page_source
#         search_bs = BeautifulSoup(search_html, 'html.parser')
#         self.n_papers = int(search_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_total_count'}).find_all('font')[1].get_text())
    
    
    def page_crawling(self):
        for view in range(2):
            self.page_filler()
            for i in range(view*20, view*20 + self.max_page_view+1):
                self.browser.find_element_by_xpath('//*[@id="ctl11_ctl01_ctl00_KMGridList"]/tbody/tr[22]/td/table/tbody/tr/td[{}]/a'.format(i)).click()
                time.sleep(3)
                self.page_filler()
            self.browser.get('http://kmbase.medric.or.kr/Main.aspx?menu=01&d=SEARCH&s=NORMAL&kw=%ub300%uc0ac%uc99d%ud6c4%uad70')
            
        
    
    def get_query_links(self, bs_object):
        query_links = bs_object.find('div',{'id' : 'mainWrapper'}).find('table', {'id' : 'ctl11_ctl01_ctl00_KMbase_table'}).find_all('li')
        return query_links
    
    def get_filtered_links(self, query_links):
        query_links_filtered =[]
        for idx, link in enumerate(query_links) :
            refined = str(link).strip('<li>').strip('</li>').strip()

            if re.compile('<a').match(refined):
                splited = refined.split('>')[0].strip('<a href=').strip('"')
                if re.compile('http').match(splited):
                    continue
                else:
                    query_links_filtered.append(splited)   
        return np.unique(query_links_filtered)
    
    def get_query_bs(self, filtered_link):
        query_addr = basic_url + filtered_link.replace('amp;', '')
        query_html = urlopen(query_addr)
        query_bs = BeautifulSoup(query_html, 'html.parser')
        journal_info = query_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_journal_info'}).select('a')[0].get('href')
        return query_bs, journal_info

    def make_keywords_str(self, keywords):
        keywords_str = ''
        for i in range(len(keywords)):
            keywords_str += keywords[i] + ','
        return keywords_str
    
    
    
    def page_filler(self):
        search_html = self.browser.page_source
        search_bs = BeautifulSoup(search_html, 'html.parser')
        search_links = get_query_links(search_bs)
        search_links_filtered = get_filtered_links(search_links)

        for i in range( len( search_links_filtered)):
            self.article_addr = basic_url + str(search_links_filtered[i]).replace('amp;', '')
            article_bs = BeautifulSoup(urlopen(self.article_addr), 'html.parser')

            title = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_title_kor'}).get_text()

            abstract_lang = 'en'
            keywords_lang = 'en'
            try : 
                if bool(article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_abstract_kor'}).get_text()):
                    abstract = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_abstract_kor'}).get_text()
                    abstract_lang = 'ko'
                else:
                    abstract = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_abstract_eng'}).get_text()

            except :
                try :
                    abstract = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_abstract_eng'}).get_text()
                except : 
                    abstract = [""]
                


            try:
                if bool(article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_keyword_kor' }).get_text()):
                    keywords = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_keyword_kor' }).get_text().split(';')
                    keywords_lang ='ko'
                else:
                    keywords = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_keyword_eng' }).get_text().split(';')

            except : 
                try : 
                    keywords = article_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_keyword_eng' }).get_text().split(';')
                    
                except :
                    keywords = [""]
                    
            keywords_str = make_keywords_str(keywords)

            self.papers.loc[self.df_idx] = title, keywords_lang, keywords_str , abstract_lang, abstract
            self.df_idx +=1
             

In [98]:
crawler = crawler()

In [105]:
crawler.page_crawling()

In [106]:
crawler.papers

,title,keywords_lang,keywords,abstract_lang,abstract
0,한국 성인의 식생활평가지수에 기반한 전체 식사의 질과 대사증후군 구성요서 및 대사증...,en,"diet quality, Korean Healthy Eating Index, met...",en,Objectives: This study examined the associatio...
1,폐경 경험 및 대사증후군 구성요소와 치주염의 관련성,en,"Menopause status, Metabolic syndrome, Oral hea...",en,Objectives: The purpose of this study was to i...
2,대사증후군 요소와 치주염의 관련성: 35-79세 한국인,en,"Epidemiology, Metabolic syndrome, Periodontitis,",en,Objectives: The purpose of this study was to i...
3,"12주간의 복합운동이 대사증후군 동반 40대 비만여성의 경동맥 내중막 두께, 혈관기...",en,"Exercise, Intima-media thickness, Metabolic sy...",en,Purpose: The purpose of this study was to inve...
4,계획된 행위이론에 근거한 대사증후군을 진단받은 대상자의 건강행위 설명모형,ko,"대사증후군, 의도, 가족의 지지, 건강행위,",en,Purpose: This study aimed to identify the fact...
...,...,...,...,...,...
375,12주간의 복합운동이 비만노인들의 신체조성과 대사증후군 위험인자에 미치는 영향,en,"Combined Exercise Program, The Obese Elderly W...",en,The purpose of this study was to examine the e...
376,근력 운동의 대사증후군 예방 효과,en,"strength exercise, aerobic exercise, metabolic...",en,Exercise treatements against metabolic syndrom...
377,축구가 지적장애 남학생의 대사증후군 위험요인과 체력에 미치는 영향,en,"Intellectual Disability Students, Physical Fit...",en,This study was performed to identify the effec...
378,"조현병, 조현정동장애 외래 환자들의 대사증후군 유병률",en,"Schizophrenia, Schizoaffective disorder, Metab...",en,Objectives: The aim of this study were to asse...
